In [90]:
import pandas as pd 
import numpy as np 
import warnings
from geneticalgorithm import geneticalgorithm as ga
from scipy.optimize import minimize

# Suppress all warnings
warnings.filterwarnings("ignore")

def minimize_loglikelihood(loglik_func, initial_guess):
    result = minimize(loglik_func, initial_guess, method='nelder-mead', args=(), tol=1e-6, options={'maxiter': 2e12})
    return result.x, result.fun

def log_likelihood_direct(xy): # loglikelihod function based on direct solution of correlation matrix 
    expected=Total_records['muij']
    observed=Total_records['yij']
    Sigma2=xy[0]
    Tau2=xy[1]
    Err=np.array(observed-expected).reshape(-1,1)
    Err_trans=np.transpose(Err)
    locator=0
    C=np.zeros((N,N))
    for k in range(M):
        n=int(events.at[k,'Train_ni'])
        for i in range(n):
            for j in range(n):
                C[locator+i][locator+j]+=Tau2
        locator+=n
    C+=Sigma2*np.identity(N)

    kc=0.1
    LN_det_C=-np.inf
    while LN_det_C==-np.inf:
        det_kC=np.linalg.det(kc*C)
        LN_det_C=np.log(det_kC)-N*np.log(kc)
        kc+=0.1

    Inv_C=np.linalg.inv(C)
    LnL=(-0.5*N*np.log(2*np.pi)-0.5*LN_det_C-0.5*np.dot(np.dot(Err_trans,Inv_C),Err))
    return -LnL[0][0]

def  log_likelihood_abrahamson(xy): 
    Sigma2=xy[0]
    Tau2=xy[1]
    ans=-0.5*N*np.log(2*np.pi)-0.5*(N-M)*np.log(Sigma2)-0.5*sum(np.log(Sigma2+events.Train_ni*Tau2))-(0.5/Sigma2)*sum((Total_records['yij']-Total_records['Y_'])**2)-0.5*sum((events.Train_ni*(events.Y_-events.Mu_)**2)/(Sigma2+events.Train_ni*Tau2))
    return -ans

# New equation
def  log_likelihood_mohammadi(xy): 
    Sigma2=xy[0]
    Tau2=xy[1]
    ans=-0.5*N*np.log(2*np.pi)-0.5*(N-M)*np.log(Sigma2)-0.5*sum(np.log(Sigma2+events.Train_ni*Tau2))-(0.5/Sigma2)*sum((Total_records['yij']-Total_records['muij'])**2)+(0.5/Sigma2)*sum(((events.Train_ni**2)*Tau2)/(Sigma2+events.Train_ni*Tau2)*(events.Y_-events.Mu_)**2)
    return -ans


# Define constants and variables
xy = (0.5,0.25)
Total_records = pd.read_csv('testLL.csv', index_col=0).dropna(axis=1, how='all')
Total_records=Total_records.iloc[0:100,:]
events = Total_records.drop_duplicates(subset=['Train_i']).reset_index(drop=True)
N = len(Total_records)
M = len(events)

# Compute log likelihood using three different functions
import time 
time1=time.time()
ll_direct = log_likelihood_direct(xy)
time2=time.time()
time_direct=time2-time1
time1=time.time()
ll_abrahamson = log_likelihood_abrahamson(xy)
time2=time.time()
time_abrahamson=time2-time1
time1=time.time()
ll_mohammadi = log_likelihood_mohammadi(xy)
time2=time.time()
time_mohammadi=time2-time1
# Print results
print("Log likelihood based on direct solution of correlation matrix: ", ll_direct, 'time= ', time_direct)
print("Log likelihood based on Abrahamson's formula: ", ll_abrahamson, 'time= ', time_abrahamson)
print("Log likelihood based on Mohammadi's equation: ", ll_mohammadi, 'time= ', time_mohammadi)

Log likelihood based on direct solution of correlation matrix:  94.6987092265451 time=  0.019453048706054688
Log likelihood based on Abrahamson's formula:  115.58904037009574 time=  0.0010581016540527344
Log likelihood based on Mohammadi's equation:  94.69870970546921 time=  0.0005035400390625


In [91]:
# Solution with Mohammadi's Equation
initial_guess = (0.5, 0.25)
result_Mohammadi = minimize_loglikelihood(log_likelihood_mohammadi, initial_guess)
print("Minimized Log likelihood based on Mohammadi's equation: ", result_Mohammadi)

Minimized Log likelihood based on Mohammadi's equation:  (array([0.25798872, 0.00881386]), 75.794181863673)


In [92]:
# Solution with Direct Equation
result_direct = minimize_loglikelihood(log_likelihood_direct, initial_guess)
print("Minimized Log likelihood based on direct solution of correlation matrix: ", result_direct)

Minimized Log likelihood based on direct solution of correlation matrix:  (array([0.25798891, 0.00881413]), 75.79418167349567)


In [93]:
# Solution with Abrahamson Equation
result_abrahamson = minimize_loglikelihood(log_likelihood_abrahamson, initial_guess)
print("Minimized Log likelihood based on Abrahamson Equation: ", result_abrahamson)

Minimized Log likelihood based on Abrahamson Equation:  (array([ 0.49432233, -0.02252083]), 104.22577547520191)
